In [1]:
import cv2
import numpy as np
import imutils
from collections import deque
import argparse
import easydict

In [2]:
cap = cv2.VideoCapture('tracking1.avi')
number=0
pts = deque(maxlen=64)
colorRanges = [((157, 0, 0), (179, 255, 255), "red")
               ,((0, 196, 106), (35, 255, 255), "yellow")
              ]


In [3]:
def nothing(x):
    pass

In [4]:
cv2.namedWindow("Trackbars")
cv2.createTrackbar("video", "Trackbars", 0,3,nothing)

In [6]:
while True:
    ret, frame = cap.read()
    v_number = cv2.getTrackbarPos("video","Trackbars")
    if number !=v_number:
        number=v_number
        cap.release()
        cap = cv2.VideoCapture('tracking'+str(number+1)+'.avi')
    if ret == False:
        cap.set(cv2.CAP_PROP_POS_FRAMES,-1)
        continue
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for (lower, upper, colorName) in colorRanges:
        mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if imutils.is_cv2() else cnts[0]
        center = None
        if cnts is None:
            continue
        if len(cnts) > 0:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            (cX, cY) = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.putText(frame, colorName, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 255, 255), 2)                
    pts.appendleft(center)
    for i in range(1, len(pts)):

        if pts[i - 1] is None or pts[i] is None:
            continue
        # otherwise, compute the thickness of the line and
        # draw the connecting lines
        thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    key= cv2.waitKey(1)
    if key==27:
        break
        
cap.release()
cv2.destroyAllWindows()